# Hypothesis Testing

From lecture, we know that hypothesis testing is a critical tool in determing what the value of a parameter could be.

We know that the basis of our testing has two attributes:

**Null Hypothesis: $H_0$**

**Alternative Hypothesis: $H_a$**

The tests we have discussed in lecture are:

* One Population Proportion
* Difference in Population Proportions
* One Population Mean
* Difference in Population Means

In this tutorial, I will introduce some functions that are extremely useful when calculating a t-statistic and p-value for a hypothesis test.

Let's quickly review the following ways to calculate a test statistic for the tests listed above.

The equation is:

$$\frac{Best\ Estimate - Hypothesized\ Estimate}{Standard\ Error\ of\ Estimate}$$ 

We will use the examples from our lectures and use python functions to streamline our tests.

In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import scipy.stats.distributions as dist

### One Population Proportion

#### Research Question 

In previous years 52% of parents believed that electronics and social media was the cause of their teenager’s lack of sleep. Do more parents today believe that their teenager’s lack of sleep is caused due to electronics and social media? 

**Population**: Parents with a teenager (age 13-18)  
**Parameter of Interest**: p  
**Null Hypothesis:** p = 0.52  
**Alternative Hypthosis:** p > 0.52 (note that this is a one-sided test)

1018 Parents

56% believe that their teenager’s lack of sleep is caused due to electronics and social media.

\begin{align*}
    H_0: p &= 0.52 \\
    H_a: p &> 0.52 \\
    \hat{p} &= 0.56 (\text{ best estimate of } p) \\
    \alpha &= 0.05 (\text{ significance level}) \\
    n &= 1018
\end{align*}

\begin{align*}
\text{test statistic} &= \frac{\text{Best estimate - Hypothesized estimate}}{\text{standard error of estimate}} \\
    &= \frac{\hat{p} - p_0}{s.e.(\hat{p})} \\
    &= \frac{\hat{p} - p_0}{\sqrt{p_0 (1-p_0)/n}}
\end{align*}

So our test statistic can be computed as follows:
\begin{align*}
    z &= \text{ "z-statistic"} \\
      &= \frac{0.56 - 0.52}{\sqrt{0.52*0.48/1018}} \\
      &= \frac{0.04}{0.0157} = 2.5555
\end{align*}

From the z-table, one deduces `p-value`: $P(Z \geq 2.555) = 1 - \Phi(2.555) = 0.0053 < 0.05 = \alpha$, thereby one rejects the null hypothesis $H_0$.

In [2]:
n = 1018
pnull = .52
phat = .56
sm.stats.proportions_ztest(phat * n, n, pnull, alternative='larger', prop_var=0.52)
# 'larger' <-> phat > pnull (which is alternative hypothesis) / prop_var = variance of population proportion(None if assuming it follows Binom Dist.)
# return val = (z-score, p-value)

(2.5545334262132955, 0.005316510991822442)

### Difference in Population Proportions

#### Research Question

Is there a significant difference between the population proportions of parents of black children and parents of Hispanic children who report that their child has had some swimming lessons?

**Populations**: All parents of black children age 6-18 and all parents of Hispanic children age 6-18  
**Parameter of Interest**: p1 - p2, where p1 = black and p2 = hispanic  
**Null Hypothesis:** p1 - p2 = 0  
**Alternative Hypthosis:** p1 - p2 $\neq$ 0  


91 out of 247 (36.8%) sampled parents of black children report that their child has had some swimming lessons.

120 out of 308 (38.9%) sampled parents of Hispanic children report that their child has had some swimming lessons.

To deal w/ the difference in population proportions using the methods below, we assume that
1. two samples are independent and
2. large enough sample sizes: $n_1\hat{p}_1, n_1(1-\hat{p}_1), n_2\hat{p}_2, n_2(1-\hat{p}_2) \geq 10$.<br>
Observe that 2. is satisfied, for:
$$
    247 \cdot 0.38 = 94, \quad 247 \cdot (1-0.38) = 153 \\
    308 \cdot 0.38 = 117, \quad 308 \cdot (1-0.38) = 191
$$
are all greater than or equal to $10$.

\begin{align*}
    H_0: p_1 - p_2 &= 0 \\
    H_a: p_1 - p_2 &\neq 0 \\
    \hat{p} &= \frac{x_1 + x_2}{n_1 + n_2} = \frac{91 + 120}{247 + 308} = 0.38 (\text{ best estimate of } p) \\
    \alpha &= 0.10 (\text{ significance level}) \\
    n_1 &= 247, x_1= 91 \\
    n_2 &= 308, x_2= 120
\end{align*}

From these, $\hat{p}_1 = 91/247 = 0.37, \quad \hat{p}_2 = 120/308 = 0.39$

\begin{align*}
\text{test statistic} &= \frac{\text{Best estimate - Hypothesized estimate}}{\text{standard error of estimate}} \\
    &= \frac{(\hat{p_1} - \hat{p_2}) - \hat{p}}{s.e.(\hat{p})} \\
    &= \frac{(\hat{p_1} - \hat{p_2}) - \hat{p}}{\sqrt{\hat{p} (1-\hat{p})(\frac{1}{n_1} + \frac{1}{n_2})}} \\
    &= \frac{-0.02}{\sqrt{0.37(1-0.37)(1/247 + 1/308)}} \\
    &= -0.48
\end{align*}

This leads to compute the p-value, which is two-sided:
$$
    \text{p-value } = 2 * \Phi(-0.48) \approx 0.63 > 0.10 = \alpha
$$

, which implies one fails to reject the null hypothesis $H_0$. Hence, No significant difference between two population proportions. But this does `NOT` mean $H_0$ should be true.

Instead, one may use $\chi^2$-test or Fisher's Exact test: the former is two-sided and the latter is one-sided hypothesis with small sample sizes.

In [3]:
# This example implements the analysis from the "Difference in Two Proportions" lecture videos

# Sample sizes
n1 = 247
n2 = 308

# Number of parents reporting that their child had some swimming lessons
y1 = 91
y2 = 120

# Estimates of the population proportions
p1 = round(y1 / n1, 2)
p2 = round(y2 / n2, 2)

# Estimate of the combined population proportion
phat = (y1 + y2) / (n1 + n2)

# Estimate of the variance of the combined population proportion
va = phat * (1 - phat)

# Estimate of the standard error of the combined population proportion
se = np.sqrt(va * (1 / n1 + 1 / n2))

# Test statistic and its p-value
test_stat = (p1 - p2) / se
pvalue = 2*dist.norm.cdf(-np.abs(test_stat))

# Print the test statistic its p-value
print("Test Statistic")
print(round(test_stat, 2))

print("\nP-Value")
print(round(pvalue, 2))

Test Statistic
-0.48

P-Value
0.63


### One Population Mean

#### Research Question 

Is the average cartwheel distance (in inches) for adults 
more than 80 inches?

**Population**: All adults  
**Parameter of Interest**: $\mu$, population mean cartwheel distance.
**Null Hypothesis:** $\mu$ = 80
**Alternative Hypthosis:** $\mu$ > 80

25 Adults

$\mu = 82.46$

$\sigma = 15.06$

Likewise, needed assumptions are:
1. simple random sample
2. normal distribution in population.

\begin{align*}
    H_0: \mu &= 80 \\
    H_a: \mu &> 80 \\
    \alpha &= 0.05 \\
    
    
    \text{test statistic} &= \frac{\text{Best estimate - null value}}{\text{estimated standard error}} \\
    &= \frac{\bar{x} - \mu}{s/\sqrt{n}} \\
    &= \frac{82.48 - 80}{15.06/\sqrt{25}} \\
    &= \frac{2.48}{3.012} \\
    &= 0.82
\end{align*}
where `(estimated standard error of the sample mean)` = $\frac{s}{\sqrt{n}}$.

If $H_0$ true, t-test statistic follows a student t-distribution w/ df(degrees of freedom) = $n-1=24$. Then using $t(24)-$ dist', the p-value is equal to $a$ such that $t_a = 0.82$, which is $a = 0.2101$:
so p-value $> 0.05 =  \alpha$, and fails to reject $H_0$.

Furthermore, 95% C.I.(Confidence Interval) estimate is given by: $\bar{x} \pm t^* (\frac{s}{\sqrt{n}}) = 0.48 \pm 1.711 \cdot (\frac{15.06}{\sqrt{25}}) = (77.33, 87.63)$. Observe that our mean $\mu = 80$ lies inside this C.I. estimate, which again emphasizes "failed to reject $H_0$".

In [4]:
df = pd.read_csv("Cartwheeldata.csv")
df.head()

,ID,Age,Gender,GenderGroup,Glasses,GlassesGroup,Height,Wingspan,CWDistance,Complete,CompleteGroup,Score
0,1,56,F,1,Y,1,62.0,61.0,79,Y,1,7
1,2,26,F,1,Y,1,62.0,60.0,70,Y,1,8
2,3,33,F,1,Y,1,66.0,64.0,85,Y,1,7
3,4,39,F,1,N,0,64.0,63.0,87,Y,1,10
4,5,27,M,2,N,0,73.0,75.0,72,N,0,4


In [5]:
n = len(df)
mean = df["CWDistance"].mean()
sd = df["CWDistance"].std()
(n, mean, sd)

(25, 82.48, 15.058552387264852)

In [6]:
sm.stats.ztest(df["CWDistance"], value = 80, alternative = "larger")

# While sm.stats.proportion_ztest() is for binomial r.v., sm.stats.ztest() if for continuous r.v.

(0.8234523266982029, 0.20512540845395266)

### Difference in Population Means

#### Research Question 

Considering adults in the NHANES data, do males have a significantly higher mean Body Mass Index than females?

**Population**: Adults in the NHANES data.  
**Parameter of Interest**: $\mu_1 - \mu_2$, Body Mass Index.  
**Null Hypothesis:** $\mu_1 = \mu_2$  
**Alternative Hypthosis:** $\mu_1 \neq \mu_2$

2976 Females 
$\mu_1 = 29.94$  
$\sigma_1 = 7.75$  

2759 Male Adults  
$\mu_2 = 28.78$  
$\sigma_2 = 6.25$  

$\mu_1 - \mu_2 = 1.16$

(Population Mean Difference)<br>
Likewise, our assumptions are:
1. random sample of differences
2. popoulation of differences is normally distributed(large sample size \geq 25)

\begin{align*}
    H_0: \mu_d &= 0 \\
    H_a: \mu_d &\neq 0 \\
    \alpha &= 0.05 \\
        
    \text{test statistic} &= \frac{\text{Best estimate - hypothesized estimate}}{\text{estimated standard error}} \\
    &= \frac{\bar{x}_d - \mu_d  }{s_d/\sqrt{n}} \\
    &= \frac{17.30 - 0}{28.49/\sqrt{20}} \\    
    &= 2.72
\end{align*}
where `(estimated standard error of the mean difference)` = $\frac{s_d}{\sqrt{n}}$.

If $H_0$ true, t-test statistic follows a student t-distribution w/ df(degrees of freedom) = $n-1=24$. Then using $t(19)-$ dist', the p-value is equal to $2 \cdot a$ such that $-t_a = 2.72$, which is $a = 0.0007$:
so p-value $ = 0.0014 > 0.05 =  \alpha$, and leads us to reject $H_0$.

Furthermore, 95% C.I.(Confidence Interval) estimate is given by: $\bar{x}_d \pm t^* (\frac{s_d}{\sqrt{n}}) = 17.30 \pm 2.093 \cdot (\frac{28.49}{\sqrt{20}}) = (3.97, 30.63)$. Observe that our mean difference $\mu_d = 0$ does lie inside this C.I. estimate, which again emphasizes "mean difference $\neq 0$".

In [7]:
url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [8]:
females = da[da["RIAGENDR"] == 2]
male = da[da["RIAGENDR"] == 1]

In [9]:
n1 = len(females)
mu1 = females["BMXBMI"].mean()
sd1 = females["BMXBMI"].std()

(n1, mu1, sd1)

(2976, 29.93994565217392, 7.753318809545674)

In [10]:
n2 = len(male)
mu2 = male["BMXBMI"].mean()
sd2 = male["BMXBMI"].std()

(n2, mu2, sd2)

(2759, 28.778072111846942, 6.2525676168014614)

In [11]:
sm.stats.ztest(females["BMXBMI"].dropna(), male["BMXBMI"].dropna())
# return (tstat, p-value)

(6.1755933531383205, 6.591544431126401e-10)